In [1]:
# Eval data link: https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/dataset_eval.zip
!curl -L -o ../data/dataset_eval.zip https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/dataset_eval.zip
!unzip -d ../data/eval ../data/dataset_eval.zip
!rm ../data/dataset_eval.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  134k  100  134k    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k
Archive:  ../data/dataset_eval.zip
  inflating: ../data/eval/answers.json  
  inflating: ../data/eval/corpus.json  
  inflating: ../data/eval/errors.json  
  inflating: ../data/eval/queries.json  
  inflating: ../data/eval/relevant_docs.json  
rm: cannot remove '../data/eval/dataset_eval.zip': No such file or directory


# Evaluation without reranker

In [ ]:
import json
from pathlib import Path
import sys
sys.path.append("../src")
from rag_alm_assistant.orchestrator import RAGOrchestrator

from bert_score import score

EVAL_DIR = Path("../data/eval")

with open(EVAL_DIR / "queries.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

with open(EVAL_DIR / "answers.json", "r", encoding="utf-8") as f:
    gold_answers = json.load(f)

with open(EVAL_DIR / "relevant_docs.json", "r", encoding="utf-8") as f:
    gold_sources = json.load(f)

uuids = list(queries.keys())
orchestrator = RAGOrchestrator(use_reranker = False, use_memory= False)

preds = []  # answer from our pipeline
refs = []   # references

for uid in uuids:
    q = queries[uid]
    ref_answer = gold_answers[uid]
    ref_sources = gold_sources[uid]

    pred_answer, pred_source = orchestrator.ask(q)

    preds.append(pred_answer)
    refs.append(ref_answer)

# Calcul BERTScore
P, R, F1 = score(
    cands=preds,
    refs=refs,
    model_type = "mistralai/Mistral-7B-Instruct-v0.3",
    lang = "fr",
    rescale_with_baseline=True
)

f1_mean = F1.mean().item()
print(f"Mean BERTScore F1: {f1_mean:.4f}  ({f1_mean*100:.2f}%)")

/workspace/RAG_ALM_assistant/notebooks/../src/rag_alm_assistant/retrieval/retriever.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, encode_kwargs=encode_kwargs)
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/workspace/RAG_ALM_assistant/notebooks/../src/rag_alm_assistant/llm.py:52: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen_pipeline)
/workspace/RAG_ALM_assistant/notebooks/../src/rag_alm_assistant/orchestrator.py:67: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationBufferMemory(
/workspace/RAG_ALM_assistant/notebooks/../src/rag_alm_assistant/orchestrator.py:111: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.q

# Evaluation with reranker

In [ ]:
import json
from pathlib import Path
import sys
sys.path.append("../src")
from rag_alm_assistant.orchestrator import RAGOrchestrator

from bert_score import score

EVAL_DIR = Path("../data/eval")

with open(EVAL_DIR / "queries.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

with open(EVAL_DIR / "answers.json", "r", encoding="utf-8") as f:
    gold_answers = json.load(f)

with open(EVAL_DIR / "relevant_docs.json", "r", encoding="utf-8") as f:
    gold_sources = json.load(f)

uuids = list(queries.keys())
orchestrator = RAGOrchestrator(use_reranker = True, use_memory=False)

preds = []  # answer from our pipeline
refs = []   # references

for uid in uuids:
    q = queries[uid]
    ref_answer = gold_answers[uid]
    ref_sources = gold_sources[uid]

    pred_answer, pred_source = orchestrator.ask(q)

    preds.append(pred_answer)
    refs.append(ref_answer)

# Calcul BERTScore
P, R, F1 = score(
    cands=preds,
    refs=refs,
    model_type = "mistralai/Mistral-7B-Instruct-v0.3",
    lang = "fr",
    rescale_with_baseline=True
)

f1_mean = F1.mean().item()
print(f"Mean BERTScore F1: {f1_mean:.4f}  ({f1_mean*100:.2f}%)")